In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import sys
sys.path.append('../')

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender
from src.myf import reduction_memory

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('../data/retail_train.csv')
data = reduction_memory(data)

item_features = pd.read_csv('../data/product.csv')
item_features = reduction_memory(item_features)

user_features = pd.read_csv('../data/hh_demographic.csv')

before:		230.09 MB
after:		141.41 MB
reduсed:	88.68 MB
before:		5.17 MB
after:		4.25 MB
reduсed:	0.92 MB


In [3]:
# column processing
# column names in lower case
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

# rename columns
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

In [4]:
# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)

val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) & 
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_train_lvl_1.copy() # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_val_lvl_1.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
2104867,2070,40618492260,594,1019940,1,1.00,311,-0.29,40,86,0.0,0.0
2107468,2021,40618753059,594,840361,1,0.99,443,0.00,101,86,0.0,0.0
2107469,2021,40618753059,594,856060,1,1.77,443,-0.09,101,86,0.0,0.0
2107470,2021,40618753059,594,869344,1,1.67,443,-0.22,101,86,0.0,0.0
2107471,2021,40618753059,594,896862,2,5.00,443,-2.98,101,86,0.0,0.0


In [5]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data=data_train_lvl_1,
                                   item_features=item_features,)
data_train_lvl_1.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0,2.99
11,1364,26984896261,1,999999,1,2.19,31742,0.00,1520,1,0.0,0.0,2.19
12,1364,26984896261,1,999999,1,2.99,31742,-0.40,1520,1,0.0,0.0,2.99
13,1364,26984896261,1,999999,1,3.09,31742,0.00,1520,1,0.0,0.0,3.09
14,1364,26984896261,1,937406,1,2.50,31742,-0.99,1520,1,0.0,0.0,2.50


In [6]:
n_items_after = data_train_lvl_1['item_id'].nunique()
print(f'Decreased # items from {n_items_before} to {n_items_after}')

Decreased # items from 83685 to 5001


In [7]:
recommender = MainRecommender(data_train_lvl_1)

In [8]:
recommender.get_als_recommendations(user=2375, N=5)

[899624, 871756, 1044078, 883932, 1106523]

In [9]:
recommender.get_own_recommendations(user=2375, N=5)

[948640, 918046, 847962, 907099, 873980]

In [10]:
recommender.get_similar_items_recommendation(user=2375, N=5)

[1046545, 1044078, 999270, 878285, 1018809]

In [11]:
recommender.get_similar_users_recommendation(user=2375, N=5)

[820612, 921406, 1101502, 841365, 10457044]

### Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [12]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns = ['user_id', 'actual']
result_lvl_1.head()

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."
2,4,"[883932, 970760, 1035676, 1055863, 1097610, 67..."
3,6,"[1024306, 1102949, 6548453, 835394, 940804, 96..."
4,7,"[836281, 843306, 845294, 914190, 920456, 93886..."


In [13]:
result_lvl_1 = result_lvl_1[result_lvl_1['user_id'].isin(data_train_lvl_1['user_id'])]
result_lvl_1.head()

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."
2,4,"[883932, 970760, 1035676, 1055863, 1097610, 67..."
3,6,"[1024306, 1102949, 6548453, 835394, 940804, 96..."
4,7,"[836281, 843306, 845294, 914190, 920456, 93886..."


In [39]:
k = 100
N = k

In [40]:
result_lvl_1['similar_items_recommendation'] = result_lvl_1['user_id'].apply(
    lambda x: recommender.get_similar_items_recommendation(user=x, N=N)
)# apply

result_lvl_1.head(2)

,user_id,actual,similar_items_recommendation,als_recommendations,own_recommendations
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[842762, 1007512, 9297615, 5577022, 1128539, 9...","[885290, 1082212, 1037332, 856942, 824758, 884...","[856942, 9297615, 5577022, 877391, 9655212, 88..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[8090537, 5569845, 1044078, 985999, 880888, 81...","[5569230, 1029743, 866211, 933835, 916122, 112...","[911974, 1076580, 1103898, 5567582, 1056620, 9..."


In [41]:
result_lvl_1['als_recommendations'] = result_lvl_1['user_id'].apply(
    lambda x: recommender.get_als_recommendations(user=x, N=N)
)# apply

result_lvl_1.head(2)

,user_id,actual,similar_items_recommendation,als_recommendations,own_recommendations
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[842762, 1007512, 9297615, 5577022, 1128539, 9...","[885290, 1082212, 1037332, 856942, 824758, 884...","[856942, 9297615, 5577022, 877391, 9655212, 88..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[8090537, 5569845, 1044078, 985999, 880888, 81...","[5569230, 1029743, 866211, 933835, 916122, 112...","[911974, 1076580, 1103898, 5567582, 1056620, 9..."


In [42]:
result_lvl_1['own_recommendations'] = result_lvl_1['user_id'].apply(
    lambda x: recommender.get_own_recommendations(user=x, N=N)
)# apply

result_lvl_1.head(2)

,user_id,actual,similar_items_recommendation,als_recommendations,own_recommendations
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[842762, 1007512, 9297615, 5577022, 1128539, 9...","[885290, 1082212, 1037332, 856942, 824758, 884...","[856942, 9297615, 5577022, 877391, 9655212, 88..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[8090537, 5569845, 1044078, 985999, 880888, 81...","[5569230, 1029743, 866211, 933835, 916122, 112...","[911974, 1076580, 1103898, 5567582, 1056620, 9..."


In [43]:
def evaluete_pred(data, true, k=5):
    algs = {}
    for col in data.columns[2:]:
        res = data.apply(lambda row: recall_at_k(row[col], row[true], k=k), axis=1).mean()
        algs[col] = res
        print(f'{col} : {res}')

In [44]:
evaluete_pred(result_lvl_1, 'actual', k=k)

similar_items_recommendation : 0.05256885273042442
als_recommendations : 0.0704446292510954
own_recommendations : 0.09604492955885034


In [45]:
result_lvl_1['similar_users_recommendation'] = result_lvl_1['user_id'].apply(
    lambda x: recommender.get_similar_users_recommendation(user=x, N=N)
)# apply

result_lvl_1.head(2)

ValueError: userid is out of bounds of the user_items matrix

### Задание 2.

Обучите модель 2-ого уровня, при этом:
    - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар
    - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2
    - Вырос ли precision@5 при использовании двухуровневой модели?

### Финальный проект

Мы уже прошли всю необходимуб теорию для финального проекта. Проект осуществляется на данных из вебинара (данные считаны в начале ДЗ).
Рекомендуем вам **начать делать проект сразу после этого домашнего задания**
- Целевая метрика - precision@5. Порог для уcпешной сдачи проекта precision@5 > 0.27%
- Будет public тестовый датасет, на котором вы сможете измерять метрику
- Также будет private тестовый датасет для измерения финального качества
- НЕ обязательно, но крайне желательно использовать 2-ух уровневые рекоммендательные системы в проекте
- Вы сдаете код проекта в виде github репозитория и csv файл с рекомендациями 